### Data 119, Spring 2023 - Version for SQLAlchemy 2.x
##### Homework 7: SQL  (10 questions, 50 points)

## Due: Friday May 19, 2023  11:59pm 

**Notice on SQLAlchemy Version**: This notebook is written for SQLAlchemy 2.0 (Jan 2023) or later.  The SQLAlchemy API for submitting queries from Pandas has changed from v1. 

**Pandas Cheat Sheet**: There are several Pandas documentation files you can find with a simple search. This is one that is short and informative: https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf

**SQL Cheat Sheet**: There are lots of tutorials and such for SQL queries.  Some of them might have syntax that is slightly different than what is supported by SQLite.   In addition to the examples in the [examples in the DS100 textbook Ch 7](https://www.textbook.ds100.org/ch/07/sql_intro.html), this site from CodeAcademy has a good summary of basic SQL statements:
https://www.codecademy.com/learn/learn-sql/modules/learn-sql-manipulation/cheatsheet



In [46]:
import sqlalchemy
print(sqlalchemy.__version__)

1.4.39


In [47]:
# Don't change this cell; just run it. 

import numpy as np

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

import pandas as pd


We are going to use SQL to analyze tables of films from IMDB. We have two databases, one excerpt from the IMDB public use tables https://developer.imdb.com/non-commercial-datasets/  and one the IMDB top 1000 database compiled by Omar Hany and published on kaggle https://www.kaggle.com/datasets/omarhanyy/imdb-top-1000 

Subsets of the public use dataset and a deduplicated version of IMDB-1000 are provided as `titles.2010.movies.csv` and `imdb-1000-six.csv`

First we are going to set up our connection to the SQLite database and remove any previous copies of our tables that may have been left in there.  Then we'll load the tables with data from .csv files.


In [48]:
# Don't change this cell; just run it.
# This cell initializes an empty database with two tables.
# For use with sqlachemy 2.0 and above

#Let's connect to our database system and clean up previous tables if they exist
import sqlalchemy
from sqlalchemy import text

# pd.read_sql takes in a parameter for a SQLite engine, which we create below
sqlite_uri = "sqlite:///imdb2010.db"

sqlite_engine = sqlalchemy.create_engine(sqlite_uri)

#start with an empty database - DROP the tables if they
#already exist in the database (from a previous run)
sql_expr = """
DROP TABLE IF EXISTS imdb;
"""
with sqlite_engine.connect() as conn:
    query = text(sql_expr) 
    result = conn.execute(query)
    

sql_expr = """
DROP TABLE IF EXISTS titles;
"""
with sqlite_engine.connect() as conn:
    query = text(sql_expr) 
    result = conn.execute(query)

#### Table Creation
Now that we have a clean database, we can create and load our two tables: `titles`, which contains information about all films from 2010,  and `imdb`, which contains information on the most popular films from throughout the history of cinema.

In [49]:
# Don't change this cell; just run it.
# This loads two csv files into the two tables in the SQL engine.

#read csv file into a dataframe
imdb_df = pd.read_csv('/Users/giovannimaya/Downloads/imdb-1000-six (1).csv')

# Populate players table
temp = imdb_df.to_sql('imdb', con= sqlite_engine, index=False)

#read csv file into a dataframe
titles_df = pd.read_csv('/Users/giovannimaya/Downloads/titles.2010.movies (1).csv', sep="\t")

# Populate salaries table
temp = titles_df.to_sql('titles', con=sqlite_engine, index=False)

#### Check the tables
Before we start, let's look at what's in our tables

In [50]:
# Well if this is the nice way to query the SQL engine
# and get the result as a dataframe, we should put this 
# in a function and our life will be easier.
def query(sql_expr):
    with sqlite_engine.begin() as conn:
       querytext = text(sql_expr) 
       df = pd.read_sql_query(querytext, conn)
    return(df)

In [51]:
# What tables do we have?  The SQL shell offers the .SHOW TABLES command, or .TABLES
# but those do not work in sqlalchemy.  

# Display the names of all the tables
query("SELECT  name  FROM  sqlite_master WHERE type='table'")  

,name
0,imdblol
1,title
2,imdb
3,titles


In [44]:
# query the SQL engine for the first 4 lines of titles
query("select * from titles")  

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0125742,movie,"FVVA: Femme, villa, voiture, argent","FVVA: Femme, villa, voiture, argent",0,2010,\N,75,\N
1,tt0146592,movie,Pál Adrienn,Pál Adrienn,0,2010,\N,136,Drama
2,tt0154039,movie,So Much for Justice!,Oda az igazság,0,2010,\N,100,"Drama,History"
3,tt0162942,movie,Children of the Green Dragon,A zöld sárkány gyermekei,0,2010,\N,89,Drama
4,tt0178691,movie,Het land van mijn ouders,Het land van mijn ouders,0,2010,\N,97,Documentary
...,...,...,...,...,...,...,...,...,...
12438,tt9851990,movie,Tikiri Suwanda,Tikiri Suwanda,0,2010,\N,\N,"Drama,Family"
12439,tt9852508,movie,Viyapath Bambara,Viyapath Bambara,0,2010,\N,120,Drama
12440,tt9875120,movie,Frostbite,Frostbite,0,2010,\N,90,Documentary
12441,tt9875242,movie,15 fotografii,15 fotografii,0,2010,\N,56,Drama


In [52]:
# query the SQL engine for the first 4 lines of imdb
query("select * from imdb limit 4")  

,Year,Title,Gross (M),Director,Metascore,Description
0,1994,The Shawshank Redemption,28.34,Frank Darabont,80.0,Two imprisoned men bond over a number of years...
1,1972,The Godfather,134.97,Francis Ford Coppola,100.0,The aging patriarch of an organized crime dyna...
2,2008,The Dark Knight,534.86,Christopher Nolan,84.0,When the menace known as the Joker wreaks havo...
3,1974,The Godfather: Part II,57.30,Francis Ford Coppola,90.0,The early life and career of Vito Corleone in ...


#### Question 1 (Basic Query, 6 points)

Write a SQL statement that returns the `titleType`,`primaryTitle`, and `runtimeMinutes` for all films in the `titles` table longer than 120 minutes, 
sorted in descending order of `runtimeMinutes`. 
HINT:  Check that the numbers are in fact larger than 120.
HINT:  You will need to look up the SQL syntax to change data types from string to int.

In [57]:
query("SELECT titleType, primaryTitle, CAST(runtimeMinutes AS INT) \
AS runtimeMinutes FROM titles WHERE CAST(runtimeMinutes AS INT) > 120 \
ORDER BY runtimeMinutes DESC")

,titleType,primaryTitle,runtimeMinutes
0,movie,How Does David Lynch Do It?,1800
1,movie,Hollywood East,1800
2,movie,The Clock,1440
3,movie,NHL: Washington Capitals 10 Greatest Games,1200
4,movie,Psychos DFLLL,780
...,...,...,...
805,movie,The Quiz Show Scandal,121
806,movie,Koottukar,121
807,movie,Burning Inside,121
808,movie,La Bohème,121


#### Question 2 (Unique values, 6 points)

Write a SQL statement that returns the number of different `primaryTitle` values. Compare this to the number of rows in the database.   What does this mean?  (Pigeon-hole principle?  At least one title...) 

In [59]:
query("SELECT COUNT(DISTINCT primarytitle) AS uniquetitles FROM titles")

,uniquetitles
0,12319


In [60]:
query("SELECT COUNT(*) FROM titles")

,COUNT(*)
0,12443


The difference in rows signifies that there are a number of repeated titles. 

#### Question 3 (Another single-table query, 6 points)
Write a SQL statement that returns the number of films, and average metascore for years between 1920 and 1940 that have at least one film with a metascore rating. 

In [61]:
query("SELECT COUNT(DISTINCT Year) AS FilmCount, AVG(Metascore) \
    AS AverageMetascore FROM imdb WHERE Year \
    BETWEEN 1920 AND 1940 AND Metascore IS NOT NULL")

,FilmCount,AverageMetascore
0,7,93.625


#### Question 4 (Summary Statistics (aggregate functions), 6 points)

Write a SQL statement that returns a single row containing the average `metascore` and `year` over all the rows in the `imdb` table.  (Note: the SQL "AVG" aggregate function is helpful here)

HINT:  Answers in the 19th century are not reasonable; find out why this query gives wrong answers and fix it.

In [62]:
query("SELECT AVG(Year) AS AvgYear, AVG(metascore) as AvgMetaScore from imdb")

,AvgYear,AvgMetaScore
0,1870.899497,82.241259


In [63]:
query("SELECT DISTINCT Year from imdb ORDER BY Year")

,Year
0,0
1,1921
2,1925
3,1926
4,1927
...,...
82,2018
83,2019
84,2020
85,I)


In [66]:
query("SELECT AVG(Year) AS AvgYear, AVG(metascore) as AvgMetaScore from imdb WHERE Year != 0")

,AvgYear,AvgMetaScore
0,1944.172324,82.251799


Q4: I created a query that showcases all the years. The average year was significantly
low so I felt this was necessary. I found a year of 0 so to fix it, all I had to 
do was select all of the years that were not equal to 0. 

#### Question 5 (Single-table query with grouping, 6 points)

Write a SQL statement that returns the `primaryTitle,` and the number rows in the `titles` database containing that title for titles that are present in two or more rows.  Some of these are duplicates, a small number may be films with the same name.


In [67]:
query("SELECT primaryTitle, COUNT(*) as rows FROM titles GROUP BY primaryTitle HAVING COUNT(*) > 1")

,primaryTitle,rows
0,Again,2
1,Aisha,2
2,Alice in Wonderland,2
3,All About My Father,2
4,All That Glitters,3
...,...,...
104,Vuelve a la vida,2
105,Wasted on the Young,2
106,Writer's Block,2
107,Yoga,2


#### Query 6 (more single-table queries)
One title seems to be in the database seven times.  Show all the rows of the `titles` table that match 
this title.  Can you guess why this title is present so many times?


In [68]:
query("SELECT * FROM titles WHERE primaryTitle = 'Ângelo de Sousa - Tudo o Que Sou Capaz'")

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt3815122,movie,Ângelo de Sousa - Tudo o Que Sou Capaz,Ângelo de Sousa - Tudo o Que Sou Capaz,0,2010,\N,60,"Biography,Documentary"
1,tt3815124,movie,Ângelo de Sousa - Tudo o Que Sou Capaz,Ângelo de Sousa - Tudo o Que Sou Capaz,0,2010,\N,60,"Biography,Documentary"
2,tt3815126,movie,Ângelo de Sousa - Tudo o Que Sou Capaz,Ângelo de Sousa - Tudo o Que Sou Capaz,0,2010,\N,60,"Biography,Documentary"
3,tt3815128,movie,Ângelo de Sousa - Tudo o Que Sou Capaz,Ângelo de Sousa - Tudo o Que Sou Capaz,0,2010,\N,60,"Biography,Documentary"
4,tt3815130,movie,Ângelo de Sousa - Tudo o Que Sou Capaz,Ângelo de Sousa - Tudo o Que Sou Capaz,0,2010,\N,60,"Biography,Documentary"
5,tt3815132,movie,Ângelo de Sousa - Tudo o Que Sou Capaz,Ângelo de Sousa - Tudo o Que Sou Capaz,0,2010,\N,60,"Biography,Documentary"
6,tt3815134,movie,Ângelo de Sousa - Tudo o Que Sou Capaz,Ângelo de Sousa - Tudo o Que Sou Capaz,0,2010,\N,60,"Biography,Documentary"


Q6: This movie appears so many times due to perhaps an error in the data collection 
or it was inputted for various occasions due to the version.

#### Question 7 (Join, 5 points)
Write a SQL statement that returns  `tconst` movie id, the `Year` and the movie title for each film that is in both `imdb` and `titles.` 



Since the `titles` table is entirely films released in 2010, write a statement that retrieves the ten films that satisfy the above and have `Year` equal to 2010.

In [77]:
query("SELECT titles.tconst, imdb.Year, titles.primaryTitle FROM imdb JOIN titles ON \
      imdb.Title = titles.primaryTitle")

,tconst,Year,primaryTitle
0,tt1375666,2010,Inception
1,tt10168078,1931,City Lights
2,tt1255953,2010,Incendies
3,tt0435761,2010,Toy Story 3
4,tt1371160,1921,The Kid
5,tt1639426,2010,Udaan
6,tt1130884,2010,Shutter Island
7,tt1411236,1979,Stalker
8,tt0892769,2010,How to Train Your Dragon
9,tt1077070,1995,Underground


In [74]:
query("SELECT titles.tconst, imdb.Year, titles.primaryTitle FROM imdb JOIN titles ON \
      imdb.Title = titles.primaryTitle WHERE imdb.Year = 2010")

,tconst,Year,primaryTitle
0,tt1375666,2010,Inception
1,tt1255953,2010,Incendies
2,tt0435761,2010,Toy Story 3
3,tt1639426,2010,Udaan
4,tt1130884,2010,Shutter Island
5,tt0892769,2010,How to Train Your Dragon
6,tt1555149,2010,Elite Squad 2: The Enemy Within
7,tt1504320,2010,The King's Speech
8,tt1188996,2010,My Name Is Khan
9,tt0947798,2010,Black Swan


#### Question 8 (3 points)
If there is a duplicated movie title in either database, we will get duplicated rows in the join.
Check for duplicated rows in the result to question 7.


In [79]:
query("SELECT primaryTitle, COUNT(*) as count FROM titles GROUP BY primaryTitle HAVING COUNT(*) > 1")

,primaryTitle,count
0,Again,2
1,Aisha,2
2,Alice in Wonderland,2
3,All About My Father,2
4,All That Glitters,3
...,...,...
104,Vuelve a la vida,2
105,Wasted on the Young,2
106,Writer's Block,2
107,Yoga,2


#### Question 9 (Grouping and Predicates, 5 points)
The `genres` field is a little ugly; it contains multiple tags.  Ignoring that ugliness for now, write an SQL query that returns the genre, number of films, and average film length for each value of `genres` with more than 100 rows.

In [93]:
query("SELECT genres as Genres, COUNT(*) AS Count, AVG(runtimeMinutes) AS AvgRuntime FROM titles GROUP BY genres HAVING COUNT(*) > 100")

,Genres,Count,AvgRuntime
0,Action,167,58.473054
1,"Biography,Documentary",163,72.006135
2,"Biography,Documentary,Drama",108,77.462963
3,"Biography,Documentary,History",101,72.891089
4,Comedy,735,75.363265
5,"Comedy,Drama",302,90.920530
6,"Comedy,Drama,Romance",175,95.280000
7,"Comedy,Romance",136,90.227941
8,Documentary,2733,58.942554
9,"Documentary,Drama",123,73.739837


#### Question 10 (Multi-table queries, 6 points)
Write a SQL statement that returns the title, year, director, and metascore for each film that appears in imdb but does not match any films in titles.  (Possible hint: look up the EXCEPT clause in SQLite - used similarily to UNION)  (Another approach is to test IS NULL).  Check that there are no films from 2010 in this list.

In [111]:
query("SELECT imdb.title, imdb.Year, imdb.Director, imdb.Metascore FROM imdb EXCEPT SELECT titles.primaryTitle , NULL AS year, NULL AS director, NULL AS metascore FROM titles WHERE Year = 2010")

,Title,Year,Director,Metascore
0,12 Angry Men,1957,Sidney Lumet,96.0
1,12 Monkeys,1995,Terry Gilliam,74.0
2,12 Years a Slave,2013,Steve McQueen,96.0
3,1917,2019,Sam Mendes,78.0
4,2001: A Space Odyssey,1968,Stanley Kubrick,84.0
...,...,...,...,...
393,Wonder,I),Stephen Chbosky,66.0
394,Yojimbo,1961,Akira Kurosawa,NaN
395,Your Name,0,Makoto Shinkai,79.0
396,Zindagi Na Milegi Dobara,2011,Zoya Akhtar,NaN


In [106]:
# If you want to play in SQL interactively, we can ask python to 
# imitate the not-very-fun sqlite interactive prompt:

from IPython.display import display
def repl():
  print ("Type 'quit' to exit.")
  while True:
    # Read input from the user
    command = input('> ')
    # Exit the loop if the user enters "quit"
    if command == 'quit':
        break
    # Execute the command and print the results
    try:
        with sqlite_engine.begin() as conn:
            df = pd.read_sql_query(text(command), conn)       
            display(df)
    except Exception as e:
        print('Error:', e)

In [ ]:
repl()
